<a href="https://colab.research.google.com/github/Ravolle/survivors_aaa_project/blob/main/%22bert_embed_baseline_ipynb%22_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Грузим данные и либы

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_parquet('./drive/MyDrive/AAA/avito_cv2vac.pq')
# ('./drive/MyDrive/AAA/course/avito_cv2vac.pq')

In [ ]:
df.head()

,User_id,Item_id,EventDate,last_resume_id,event_name,Region,City,session_hash,microcat_name,Platform_id,res_title,res_des,vac_title,vac_des
0,372831461,1410897751247,2023-03-13 16:01:07,287948381,Пользовательские события / Объявления / Просмо...,Санкт-Петербург,Санкт-Петербург,7813611419044570652,Полный день,2,Ищу работу,"Ищу работу технологом,нач.смены.Опыт работы 3 ...",Начальник охраны объекта,Обязанности: контроль работы постов охраны​ по...
1,366472774,768439750232,2023-03-13 14:56:13,293106495,In App Calls / Контакт IAC (в различных сценар...,Вологодская область,Череповец,2800082180641601867,Полный день,4,Ищу работу,м/ч 25л образование среднее профессиональное(п...,"Водитель, монтажник",Только с желанием работать и зарабатывать\nТрe...
2,366669357,1429452751612,2023-03-13 19:57:56,317525264,Пользовательские события / Объявления / Просмо...,Самарская область,Самара,6266140651757332598,Сменный график,3,Ищу подработку,"ищу работу, подработку с 9,00-17,00, с возможн...",Медицинская сестра в клинику эстетической меди...,Медицинская сестра в клинику эстетической меди...
3,370281435,1422666754936,2023-03-13 19:03:54,323785715,Пользовательские события / Объявления / Просмо...,Санкт-Петербург,Колпино,6498496299285393322,Сменный график,3,Для желающих подработать на дому,Без вложения средств. на компьютере.подробност...,Оператор станции обезвоживания сырого осадка,— Весь технологический процесс станции обезвож...
4,370281435,689631256782,2023-03-13 18:42:42,323785715,Пользовательские события / Объявления / Просмо...,Санкт-Петербург,Санкт-Петербург,6498496299285393322,Сменный график,3,Для желающих подработать на дому,Без вложения средств. на компьютере.подробност...,Дворник на АЗС (62),Требуется дворник для содержания территории в ...


Создадим фичу "кластер", чтобы при поиске подходящей вакансии в базе не перебирать все подряд, а только мэтч по этому полю

Если работа удаленная, регион и город не важны

In [ ]:
df['microcat_name'].unique()

array(['Полный день', 'Сменный график', 'Удалённая работа',
       'Свободный график', 'Неполный день', 'Вахтовый метод',
       'Удаленная работа', 'Вакансии'], dtype=object)

In [ ]:
df.loc[(df['microcat_name'] != 'Удаленная работа') | (df['microcat_name'] != 'Удалённая работа'), 'cluster'] = df['Region'] + df['City'] + df['microcat_name']
df.loc[(df['microcat_name'] == 'Удаленная работа') & (df['microcat_name'] == 'Удалённая работа'), 'cluster'] = 'Удаленная работа'

In [ ]:
df


,User_id,Item_id,EventDate,last_resume_id,event_name,Region,City,session_hash,microcat_name,Platform_id,res_title,res_des,vac_title,vac_des,cluster
0,372831461,1410897751247,2023-03-13 16:01:07,287948381,Пользовательские события / Объявления / Просмо...,Санкт-Петербург,Санкт-Петербург,7813611419044570652,Полный день,2,Ищу работу,"Ищу работу технологом,нач.смены.Опыт работы 3 ...",Начальник охраны объекта,Обязанности: контроль работы постов охраны​ по...,Санкт-ПетербургСанкт-ПетербургПолный день
1,366472774,768439750232,2023-03-13 14:56:13,293106495,In App Calls / Контакт IAC (в различных сценар...,Вологодская область,Череповец,2800082180641601867,Полный день,4,Ищу работу,м/ч 25л образование среднее профессиональное(п...,"Водитель, монтажник",Только с желанием работать и зарабатывать\nТрe...,Вологодская областьЧереповецПолный день
2,366669357,1429452751612,2023-03-13 19:57:56,317525264,Пользовательские события / Объявления / Просмо...,Самарская область,Самара,6266140651757332598,Сменный график,3,Ищу подработку,"ищу работу, подработку с 9,00-17,00, с возможн...",Медицинская сестра в клинику эстетической меди...,Медицинская сестра в клинику эстетической меди...,Самарская областьСамараСменный график
3,370281435,1422666754936,2023-03-13 19:03:54,323785715,Пользовательские события / Объявления / Просмо...,Санкт-Петербург,Колпино,6498496299285393322,Сменный график,3,Для желающих подработать на дому,Без вложения средств. на компьютере.подробност...,Оператор станции обезвоживания сырого осадка,— Весь технологический процесс станции обезвож...,Санкт-ПетербургКолпиноСменный график
4,370281435,689631256782,2023-03-13 18:42:42,323785715,Пользовательские события / Объявления / Просмо...,Санкт-Петербург,Санкт-Петербург,6498496299285393322,Сменный график,3,Для желающих подработать на дому,Без вложения средств. на компьютере.подробност...,Дворник на АЗС (62),Требуется дворник для содержания территории в ...,Санкт-ПетербургСанкт-ПетербургСменный график
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232814,741021250053,1427757755588,2023-03-13 15:50:07,1432203250045,Чат / События полученные через AMQP / Отправка...,Москва,Москва,5616536396553863727,Вахтовый метод,3,Няня для ребенка с проживанием,"Добрая, спокойная, уравновешенная, неконфликтн...",Требуется: Няня-гувернантка,"Требуется: Няня-гувернантка, Московская област...",МоскваМоскваВахтовый метод
232815,313382250061,1421789250018,2023-03-13 12:20:20,1432203750857,Пользовательские события / Объявления / Просмо...,Забайкальский край,Чита,6283812260717342948,Свободный график,3,"Разнорабочий, слесарь, гидромониторщик",Спакойный трудолюбивый,Водитель-Развозчик Забытые ключи,Водитель на личном транспорте. Подработка или ...,Забайкальский крайЧитаСвободный график
232816,442660750128,1432056750361,2023-03-13 23:49:45,1432209500911,Чат / События полученные через AMQP / Отправка...,Москва,Москва,8199703742503214578,Свободный график,4,Маркетолог,"Работаю в продажах , сейчас закончила обучение...",Подработка в рекламной компании,"Привет, мы ищем сотрудника готового совмещать ...",МоскваМоскваСвободный график
232817,442660750128,1430745502106,2023-03-13 23:41:06,1432209500911,Чат / События полученные через AMQP / Отправка...,Москва,Москва,8199703742503214578,Полный день,4,Маркетолог,"Работаю в продажах , сейчас закончила обучение...",Бренд менеджер полного цикла,Добрый день! Наша компания занимается рознично...,МоскваМоскваПолный день


Эта фича хорошо балансирует датасет

Вместо полного перебора мы максимум переберем 6.5% 

In [ ]:
df['cluster'].value_counts(normalize=True)

МоскваМоскваПолный день                        0.064849
МоскваМоскваСменный график                     0.038450
МоскваМоскваВахтовый метод                     0.033013
Санкт-ПетербургСанкт-ПетербургПолный день      0.026695
МоскваМоскваСвободный график                   0.026338
                                                 ...   
Волгоградская областьСуровикиноПолный день     0.000004
Ростовская областьМорозовскСменный график      0.000004
Новосибирская областьКоченевоПолный день       0.000004
Алтайский крайСибирскийВахтовый метод          0.000004
Ульяновская областьНовая МалыклаПолный день    0.000004
Name: cluster, Length: 5312, dtype: float64

### Идеи для поиска:

0. Бейзлайн на частотном анализе (типа подсчет уникальных слов)
1. Эмбеддинги Берта --> Расстояние
2. Сиамские сети --> Расстояние
3. Triplet Loss (?)
4. ALS и фишки про рекомендации (?)

### Бейзлайн на эмбеддингах ruBERT'а

In [ ]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 59.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 21.2 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from scipy import spatial
import random

Возьмем BERT'а для текстов на русском от DeepPavlov'а (можно пробовать другие варианты)

In [ ]:
rubert_tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")

rubert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased")

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from tqdm import tqdm

In [ ]:
class EmbedBaseline:
  def __init__(self, model, tokenizer):
    self.tokenizer = tokenizer
    self.model = model
  
  def get_bert_embed(self, text):
    text = "[CLS] " + text + " [SEP]"
    tokenized_text = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')

    with torch.no_grad():
        model_output = self.model(**{k: v.to(self.model.device) for k, v in tokenized_text.items()})
    
    embeddings = model_output.last_hidden_state[:, 0, :]
    
    # нормализуем
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

  
  def calculate_cosine_similarity(self, vac_des, cv_des):
    vac_vec = self.get_bert_embed(vac_des)
    cv_vec = self.get_bert_embed(cv_des)

    cosine_similarity = 1 - float(spatial.distance.cosine(vac_vec, cv_vec))

    return cosine_similarity

  
  def get_top_n_cv(self, vac_des, vac_cluster, df, top_n=5):
    res = {}
    possible_cv = df.loc[df['cluster'] == vac_cluster, 'res_des'].values[:100]

    for cv_des_ind in tqdm(range(len(possible_cv))):
      cv_des = possible_cv[cv_des_ind]
      res[cv_des] = self.calculate_cosine_similarity(vac_des, cv_des)

    return sorted(res.items(), key=lambda x: -x[1])[:top_n]    


In [ ]:
baseline_model = EmbedBaseline(rubert, rubert_tokenizer)

In [ ]:
first_vac_top5 = baseline_model.get_top_n_cv(df['vac_des'][0], df.loc[0, 'cluster'], df)

100%|██████████| 100/100 [01:18<00:00,  1.27it/s]
